# Model creation stage III (4x8)

In [1]:
%load_ext autoreload
%autoreload 2

# Setup

In [2]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from tools import common_functions as cf
from tools import regression as reg
from tools import plotting_functions as pf

from tools import sensor_gui as sg

import os
import dill

ARRAY_SIZE = 32

In [3]:
save_path = os.path.join(os.getcwd(), '..', 'models', "4x8")
models = []
        
with open(os.path.join(save_path, "stageII"), 'rb') as f:
    stageIImodel = dill.load(f)

In [4]:
import numpy as np

def numpy_to_c_array(np_array, array_name):
    """
    Convert a NumPy array to a C array representation.
    
    Parameters:
    - np_array: NumPy array to be converted
    - array_name: Name of the C array
    
    Returns:
    - A string representing the C code for the array
    """
    if not isinstance(np_array, np.ndarray):
        raise TypeError("Input should be a NumPy array")
    
    array_type = np_array.dtype
    if array_type == np.int32 or array_type == np.int64:
        c_type = 'int'
    elif array_type == np.float32:
        c_type = 'float'
    elif array_type == np.float64:
        c_type = 'double'
    else:
        raise ValueError(f"Unsupported dtype: {array_type}")
    
    # Flatten the array and format it
    flattened_array = np_array.flatten()
    array_elements = ", ".join(map(str, flattened_array))
    
    # Determine the shape of the array for multi-dimensional arrays
    shape = np_array.shape
    if len(shape) == 1:
        c_array = f"{c_type} {array_name}[] = {{{array_elements}}};"
    else:
        dimensions = "][".join(map(str, shape))
        c_array = f"{c_type} {array_name}[{dimensions}] = {{{array_elements}}};"
    
    return c_array

In [5]:
def transform_to_c_strings(expressions):
    result = []
    
    for expression in expressions:
        terms = expression.split()  # Split the expression into terms
        transformed_terms = []
        
        for term in terms:
            if '^' in term:
                base, exponent = term.split('^')
                transformed_terms.extend([base] * int(exponent))
            else:
                transformed_terms.append(term)
        
        result.append(' * '.join(transformed_terms))
    
    return result

# Example usage:
expressions = ['1', 'x0', 'x1', 'x2', 'x0^2', 'x0 x1', 'x0 x2', 'x1^2', 'x1 x2', 'x2^2', 
               'x0^3', 'x0^2 x1', 'x0^2 x2', 'x0 x1^2', 'x0 x1 x2', 'x0 x2^2', 'x1^3', 
               'x1^2 x2', 'x1 x2^2', 'x2^3']

transformed = transform_to_c_strings(expressions)
for expr in transformed:
    print(expr)


1
x0
x1
x2
x0 * x0
x0 * x1
x0 * x2
x1 * x1
x1 * x2
x2 * x2
x0 * x0 * x0
x0 * x0 * x1
x0 * x0 * x2
x0 * x1 * x1
x0 * x1 * x2
x0 * x2 * x2
x1 * x1 * x1
x1 * x1 * x2
x1 * x2 * x2
x2 * x2 * x2


# Conversion to C

In [6]:
MODEL_DEGREE = 1

for i in range(ARRAY_SIZE):

    with open(os.path.join(save_path, f"taxel{i}_linear"), 'rb') as f:
        model = dill.load(f)
        models.append(model)

In [7]:
output = ""

# Taxel coefs
for i in range(ARRAY_SIZE):
    output += numpy_to_c_array(models[i].steps[1][1].coef_.astype(np.float32), f"taxel{i}_coef") + "\n"
output += "\n"

# Coeff list
output += "float* coef_pointers[] = {"
for i in range(ARRAY_SIZE):
    output += f"&taxel{i}_coef[0][0], "
output = output[:-2] + "};\n\n"

# Second stage matrix. Bias and matrix
output += numpy_to_c_array(stageIImodel.bias.astype(np.float32), "stageII_bias") + "\n"
output += numpy_to_c_array(stageIImodel.M.astype(np.float32), "stageII_Kinv") + "\n\n"

# Now function to generate polynomial featuers
feature_names = transform_to_c_strings(models[0].steps[0][1].get_feature_names_out())
output += f"\nvoid generate_polynomial_features(float X0, float Y0, float Z0, float* output) {'{'}\n"
for i, feature in enumerate(feature_names):
    output += f"    output[{i}] = {feature};\n"
output += "}\n"

# Now function to apply the model
output += f"\nvoid apply_model(float* coefs, float* features, float* output) {'{'}\n"
output += "    for (int i = 0; i < 3; i++) {\n"
output += "        output[i] = 0;\n"
output += f"        for (int j = 0; j < {len(feature_names)}; j++) {'{'}\n"
output += f"            output[i] += coefs[i * {len(feature_names)} + j] * features[j];\n"
output += "        }\n"
output += "    }\n"
output += "}\n"

output += '\n'
output += "void apply_stage_II(float* input, float* output) {\n"
output += "    for (int i = 0; i < 96; i++) {\n"
output += "        output[i] = 0;\n"
output += "        for (int j = 0; j < 96; j++) {\n"
output += "            output[i] += stageII_Kinv[i][j] * (input[j] - stageII_bias[j]);\n"
output += "        }\n"
output += "    }\n"
output += "}\n"

print(output)

float taxel0_coef[3][4] = {-0.07463339, 0.0012431467, 0.00013812781, 85197.586, 0.023724135, -7.345255e-05, 0.0012122467, -24167.91, -0.54419154, -0.0011049075, -0.00012443142, 255035.45};
float taxel1_coef[3][4] = {-0.16767238, 0.0016198201, 0.00026701853, 86234.94, -0.10850413, -0.0002463547, 0.002676444, 4239.9204, -0.28427088, -0.0017791588, -0.00033001715, 212382.95};
float taxel2_coef[3][4] = {-0.02260424, 0.0011841796, 0.00017215461, 65904.51, -0.07032778, -0.00011107781, 0.0019061716, 18193.453, -0.8824275, -0.0015202177, -0.00087898015, 171942.66};
float taxel3_coef[3][4] = {-0.04425349, 0.0014245408, 0.00020450991, 83518.68, -0.011851424, -0.0002886133, 0.0015088418, 104045.34, -0.32043627, -0.0011459289, -0.00034598293, 447570.25};
float taxel4_coef[3][4] = {-0.043852095, 0.001691648, 9.665195e-05, -99202.836, -0.18843988, -0.00011049982, 0.0011460844, -203690.34, 0.3055662, -0.00027165966, 0.00030750848, 809537.25};
float taxel5_coef[3][4] = {0.03691216, 0.0048681786, 0.000